## Train Classifiers

In [24]:
import pandas as pd
import re
import string
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#!pip install sklearn

In [30]:
spam_ham = pd.read_csv("C:\\Users\\Thiag\\spam_ham\\data\\dataframe_to_modeling\\spam_ham.csv")

In [31]:
spam_ham.head()

,ID,Message,SPAM,Spam,text_norm,text_tokenized,text_lemmatized,Message_len,punct%
0,615317,Linda msg!,True,1,linda msg,"['linda', 'msg']","['linda', 'msg']",9,11.1
1,855344,To fazendo batida de kwui com coco,True,1,fazendo batida kwui coco,"['fazendo', 'batida', 'kwui', 'coco']","['fazendo', 'batida', 'kwui', 'coco']",28,0.0
2,1007222,Esse dinheiro só vai sair para quem está com o nome na lista\nEsse dinheiro só vai sair para que...,True,1,esse dinheiro sair nome lista esse dinheiro sair nome lista,"['esse', 'dinheiro', 'sair', 'nome', 'lista', 'esse', 'dinheiro', 'sair', 'nome', 'lista']","['esse', 'dinheiro', 'sair', 'nome', 'lista', 'esse', 'dinheiro', 'sair', 'nome', 'lista']",113,7.1
3,811063,Isso msm Irmã.. nao lembrava dos nomes kkk,True,1,isso msm irma lembrava nomes kkk,"['isso', 'msm', 'irma', 'lembrava', 'nomes', 'kkk']","['isso', 'msm', 'irma', 'lembrava', 'nome', 'kkk']",35,5.7
4,913944,Ai xara eu sujiro esse,True,1,xara sujiro,"['xara', 'sujiro']","['xara', 'sujiro']",18,0.0


In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb

ModuleNotFoundError: No module named 'feature_extraction'